# Task 1

### Importing libraries

In [1]:
# Import packages
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

### Creating the database schema

In [2]:
# Create new database (Will be saved at where the python file is saved at)
# Connect to the SQLite database using the connect function, which returns a connection object
# and then create a cursor object which allow us to execute SQL queries on a database
conn = sqlite3.connect('DatabaseSchema.db')  
c = conn.cursor()

# Create table - codebook
c.execute('''CREATE TABLE codebook
             ([id] INTEGER PRIMARY KEY, [question_id] VARCHAR, [value] INTEGER, [label] VARCHAR)''')   
# Create table - questions_table
c.execute('''CREATE TABLE questions_table
             ([id] VARCHAR PRIMARY KEY, [question_text] TEXT)''') 
# Create table - respondents_table
c.execute('''CREATE TABLE respondents_table
             ([row_id] INTEGER PRIMARY KEY AUTOINCREMENT, [id] INTEGER , [study_id] INTEGER, [sex] INTEGER, [date] DATE)''')   
# Create table - responses_table
c.execute('''CREATE TABLE responses_table
             ([id] INTEGER PRIMARY KEY AUTOINCREMENT, [respondent_id] INTEGER, [study_id] INTEGER, [question_id] VARCHAR, [response] INTEGER)''')  
# Create table - study_101
c.execute('''CREATE TABLE study_101
             ([respondent_id], [Q1], [Q2], [Q3], [Q4], [Q5], [Sex], [date])''')   
# Create table - study_102
c.execute('''CREATE TABLE study_102
             ([respondent_id], [Q1], [Q2], [Q3], [Q5], [Sex], [Number of children], [date])''')

# Close the cursor and connection objects
c.close()
conn.close()

### Importing csv file

In [3]:
conn = sqlite3.connect('DatabaseSchema.db')  
c = conn.cursor()

# Insert the values from the csv file into table - codebook
codebook = pd.read_csv (r'C:\codebook.csv',encoding = "ISO-8859-1")
codebook.to_sql('codebook', conn, if_exists='append', index = False) 

# Insert the values from the csv file into table - questions_table
questions_table = pd.read_csv (r'C:\questions_table.csv')
questions_table.to_sql('questions_table', conn, if_exists='append', index = False)

# Insert the values from the csv file into table - study_101
study_101 = pd.read_csv (r'C:\study_101.csv')
study_101.to_sql('study_101', conn, if_exists='append', index = False)

# Insert the values from the csv file into table - study_102
study_102 = pd.read_csv (r'C:\study_102.csv')
study_102.to_sql('study_102', conn, if_exists='append', index = False)

c.close()
conn.close()

### Extracting the records for insertion

In [4]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
study1 = pd.read_sql_query("SELECT * ,CASE WHEN Q3 LIKE '%1%' THEN '1' ELSE '0' END AS Q3_1, \
                            CASE WHEN Q3 LIKE '%2%' THEN '1' ELSE '0' END AS Q3_2,\
                            CASE WHEN Q3 LIKE '%3%' THEN '1' ELSE '0' END AS Q3_3,\
                            CASE WHEN Q3 LIKE '%4%' THEN '1' ELSE '0' END AS Q3_4,\
                            CASE WHEN Q3 LIKE '%5%' THEN '1' ELSE '0' END AS Q3_5,\
                            CASE WHEN Q3 LIKE '%6%' THEN '1' ELSE '0' END AS Q3_6 FROM study_101", engine)

study2 = pd.read_sql_query("SELECT * ,CASE WHEN Q3 LIKE '%1%' THEN '1' ELSE '0' END AS Q3_1, \
                            CASE WHEN Q3 LIKE '%2%' THEN '1' ELSE '0' END AS Q3_2,\
                            CASE WHEN Q3 LIKE '%3%' THEN '1' ELSE '0' END AS Q3_3,\
                            CASE WHEN Q3 LIKE '%4%' THEN '1' ELSE '0' END AS Q3_4,\
                            CASE WHEN Q3 LIKE '%5%' THEN '1' ELSE '0' END AS Q3_5,\
                            CASE WHEN Q3 LIKE '%6%' THEN '1' ELSE '0' END AS Q3_6 FROM study_102", engine)

# Print the DataFrame
print(study1)
print(study2)

   respondent_id  Q1  Q2       Q3  Q4  Q5  Sex        date Q3_1 Q3_2 Q3_3  \
0              1   3   1     2; 5   1   2    1  2018-01-02    0    1    0   
1              2   3   4     1; 2   0   1    2  2018-01-02    1    1    0   
2              3   5   2        2   0   2    1  2018-01-02    0    1    0   
3              4   3   5        6   0   1    1  2018-01-02    0    0    0   
4              5   5   1  3; 4; 5   1   1    2  2018-01-02    0    0    1   
5              6   2   4     1; 5   1   1    1  2018-01-02    1    0    0   
6              7   2   6        3   1   1    2  2018-01-02    0    0    1   
7              8   3   3        4   1   1    1  2018-01-02    0    0    0   
8              9   1   5     1; 3   1   1    1  2018-01-02    1    0    1   
9             10   1   2     1; 2   0   1    2  2018-01-02    1    1    0   

  Q3_4 Q3_5 Q3_6  
0    0    1    0  
1    0    0    0  
2    0    0    0  
3    0    0    1  
4    1    1    0  
5    0    1    0  
6    0    0    0  


### Processing data for insertion

In [5]:
# Add a new column
study1['study_id'] = 101

# Records to be inserted to table - respondents_table
study1a = study1[['respondent_id', 'study_id', 'Sex', 'date']]

# Renaming column name to be in line with table - respondents_table
study1a = study1a.rename(columns={"respondent_id": "id"})

# Records to be inserted to table - responses_table
study1b = study1.melt(id_vars=['respondent_id', 'study_id'],value_vars=['Q1', 'Q2', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q3_5', 'Q3_6', 'Q4', 'Q5'],
          var_name=['question_id'], value_name='response')

# Sorting by respondent_id and question_id 
study1b = study1b.sort_values(by=['respondent_id','question_id'])

In [6]:
study1a

,id,study_id,Sex,date
0,1,101,1,2018-01-02
1,2,101,2,2018-01-02
2,3,101,1,2018-01-02
3,4,101,1,2018-01-02
4,5,101,2,2018-01-02
5,6,101,1,2018-01-02
6,7,101,2,2018-01-02
7,8,101,1,2018-01-02
8,9,101,1,2018-01-02
9,10,101,2,2018-01-02


In [7]:
study1b

,respondent_id,study_id,question_id,response
0,1,101,Q1,3
10,1,101,Q2,1
20,1,101,Q3_1,0
30,1,101,Q3_2,1
40,1,101,Q3_3,0
50,1,101,Q3_4,0
60,1,101,Q3_5,1
70,1,101,Q3_6,0
80,1,101,Q4,1
90,1,101,Q5,2


In [8]:
# Add a new column
study2['study_id'] = 102

# Changing the date format
study2['date'] = pd.to_datetime(study2['date']).dt.strftime('%Y-%m-%d')

# Records to be inserted to table -  respondents_table
study2a = study2[['respondent_id', 'study_id', 'Sex', 'date', 'Number of children']]

# Renaming column name to be in line with table - respondents_table
study2a = study2a.rename(columns={'Number of children':'NumberOfChildren', "respondent_id": "id"})

# Records to be inserted to table - responses_table
study2b = study2.melt(id_vars=['respondent_id', 'study_id'],value_vars=['Q1', 'Q2', 'Q3_1', 'Q3_2', 'Q3_3', 'Q3_4', 'Q3_5', 'Q3_6', 'Q4', 'Q5'],
          var_name=['question_id'], value_name='response')

# Sorting by respondent_id and question_id 
study2b = study2b.sort_values(by=['respondent_id','question_id'])

In [9]:
study2a

,id,study_id,Sex,date,NumberOfChildren
0,1,102,2,2018-03-01,2
1,2,102,2,2018-03-01,5
2,3,102,1,2018-03-01,2
3,4,102,1,2018-03-01,0
4,5,102,2,2018-03-01,3
5,6,102,2,2018-03-01,0
6,7,102,2,2018-03-01,1
7,8,102,2,2018-03-01,3
8,9,102,1,2018-03-01,0
9,10,102,1,2018-03-01,2


In [10]:
study2b

,respondent_id,study_id,question_id,response
0,1,102,Q1,1
10,1,102,Q2,4
20,1,102,Q3_1,1
30,1,102,Q3_2,1
40,1,102,Q3_3,0
50,1,102,Q3_4,1
60,1,102,Q3_5,1
70,1,102,Q3_6,0
80,1,102,Q4,NaN
90,1,102,Q5,0


### Add a new column to table -  respondents_table

In [11]:
conn = sqlite3.connect('DatabaseSchema.db')
c = conn.cursor() 

# Add a new column 'NumberOfChildren' to table - respondents_table
c.execute('''ALTER TABLE respondents_table ADD COLUMN NumberOfChildren INTEGER''')

c.close()
conn.close()

### Inserting the processed records to the tables

In [12]:
conn = sqlite3.connect('DatabaseSchema.db') 
c = conn.cursor()

# Insert the values from the dataframe to table - 'respondents_table' and 'responses_table'
study1a.to_sql('respondents_table', conn, if_exists='append', index=False)
study1b.to_sql('responses_table', conn, if_exists='append', index=False)
study2a.to_sql('respondents_table', conn, if_exists='append', index=False)
study2b.to_sql('responses_table', conn, if_exists='append', index=False)

c.close()
conn.close()

### Deleting table 'study_101' and 'study_102' from the database

In [13]:
conn = sqlite3.connect('DatabaseSchema.db') 
c = conn.cursor()

# Delete from the database table - 'study_101' and 'study_102'
c.execute('''DROP TABLE study_101''')
c.execute('''DROP TABLE study_102''')

c.close()
conn.close()

### Verifying records in the tables

In [14]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
respondents_table = pd.read_sql_query("SELECT * FROM respondents_table", engine)
responses_table = pd.read_sql_query("SELECT * FROM responses_table", engine)

# Print the dataframe
print(respondents_table)
print(responses_table)

    row_id  id  study_id  sex        date  NumberOfChildren
0        1   1       101    1  2018-01-02               NaN
1        2   2       101    2  2018-01-02               NaN
2        3   3       101    1  2018-01-02               NaN
3        4   4       101    1  2018-01-02               NaN
4        5   5       101    2  2018-01-02               NaN
5        6   6       101    1  2018-01-02               NaN
6        7   7       101    2  2018-01-02               NaN
7        8   8       101    1  2018-01-02               NaN
8        9   9       101    1  2018-01-02               NaN
9       10  10       101    2  2018-01-02               NaN
10      11   1       102    2  2018-03-01               2.0
11      12   2       102    2  2018-03-01               5.0
12      13   3       102    1  2018-03-01               2.0
13      14   4       102    1  2018-03-01               0.0
14      15   5       102    2  2018-03-01               3.0
15      16   6       102    2  2018-03-0

# Task 2

### 2a) Number of people responded ‘strongly disagree’ or ‘disagree’ in Q1

In [15]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT COUNT(CASE WHEN question_id = 'Q1' AND response in ('1','2') THEN 1 END) AS \
                        [Strongly Disagree or Disagree] FROM responses_table", engine)

# Print the dataframe
print(df)

   Strongly Disagree or Disagree
0                              6


### 2b) Number of people of each sex responded ‘strongly disagree’ or ‘disagree' in Q1

In [16]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT sex, COUNT(CASE WHEN question_id = 'Q1' AND response in ('1','2') THEN 1 END) AS \
                        [Strongly Disagree or Disagree] FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id \
                        AND respondents_table.study_id=responses_table.study_id GROUP BY sex", engine)

# Print the dataframe
print(df)

   sex  Strongly Disagree or Disagree
0    1                              2
1    2                              4


### 2c) Number of people of each sex responded (i) ‘strongly disagree’ or ‘disagree’ (ii) ‘neutral’ (iii) ‘strongly agree’ or ‘agree’ in Q1

In [17]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT sex, COUNT(CASE WHEN question_id = 'Q1' AND response in ('1','2') THEN 1 END) AS [Strongly Disagree or Disagree], \
                          COUNT(CASE WHEN question_id = 'Q1' AND response in ('3') THEN 1 END) AS Neutral, \
                          COUNT(CASE WHEN question_id = 'Q1' AND response in ('4','5') THEN 1 END) AS [Strongly Agree or Agree] \
                        FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id AND \
                            respondents_table.study_id=responses_table.study_id GROUP BY sex", engine)

# Print the dataframe
print(df)

   sex  Strongly Disagree or Disagree  Neutral  Strongly Agree or Agree
0    1                              2        3                        5
1    2                              4        1                        4


### 2d) Proportion of people selected each option in Q3

In [18]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT x.question_id, x.cnt* 100.0/y.cnt_total AS [proportion(%)] \
                           FROM (SELECT question_id, count(row_id) AS cnt \
                                    FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id     \
                                                            AND respondents_table.study_id=responses_table.study_id \
                                        WHERE question_id between 'Q3_1' and 'Q3_6' and response = 1 group by 1) AS x, \
                                (SELECT count(distinct(row_id)) AS cnt_total \
                                    FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id \
                                                            AND respondents_table.study_id=responses_table.study_id) AS y", engine)
# Print the dataframe
print(df)

  question_id  proportion(%)
0        Q3_1           55.0
1        Q3_2           65.0
2        Q3_3           35.0
3        Q3_4           40.0
4        Q3_5           55.0
5        Q3_6           10.0


### 2e) Proportion of people of each sex selected each option in Q3

In [19]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT sex, x.question_id, x.cnt* 100.0/y.cnt_total AS [proportion(%)] \
                           FROM (SELECT sex, question_id, count(row_id) AS cnt \
                                    FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id     \
                                                            AND respondents_table.study_id=responses_table.study_id \
                                        where question_id between 'Q3_1' and 'Q3_6' and response = 1 group by 1,2) AS x, \
                                (SELECT count(distinct(row_id)) AS cnt_total \
                                    FROM responses_table JOIN respondents_table ON respondents_table.id=responses_table.respondent_id \
                                                            AND respondents_table.study_id=responses_table.study_id) AS y", engine)
# Print the dataframe
print(df)

    sex question_id  proportion(%)
0     1        Q3_1           25.0
1     1        Q3_2           30.0
2     1        Q3_3           15.0
3     1        Q3_4           20.0
4     1        Q3_5           30.0
5     1        Q3_6            5.0
6     2        Q3_1           30.0
7     2        Q3_2           35.0
8     2        Q3_3           20.0
9     2        Q3_4           20.0
10    2        Q3_5           25.0
11    2        Q3_6            5.0


### 2f) Number of people who had (i) no children, (ii) 1-2 children, and (iii) 3 or more children in study_102

In [20]:
# Create an engine instance
engine = create_engine('sqlite:///DatabaseSchema.db')

# Execute query and store records in the dataframe
df = pd.read_sql_query("SELECT COUNT(CASE WHEN study_id = 102 AND NumberOfChildren = 0 THEN 1 END) AS [No Children], \
                          COUNT(CASE WHEN study_id = 102 AND NumberOfChildren BETWEEN 1 AND 2 THEN 1 END) AS [1-2 Children],              \
                          COUNT(CASE WHEN study_id = 102 AND NumberOfChildren >= 3 THEN 1 END) AS [3 or more Children] \
                        FROM respondents_table", engine)

# Print the dataframe
print(df)

   No Children  1-2 Children  3 or more Children
0            3             4                   3
